In [1]:
import pandas as pd
df = pd.read_json('Sarcasm_Headlines_Dataset.json',lines=True)

E:\anaconda\envs\tfbert\lib\site-packages\pyarrow\compute.py:198: RuntimeWarning: Python binding for CumulativeSumOptions not exposed
  .format(class_name), RuntimeWarning)
E:\anaconda\envs\tfbert\lib\site-packages\pyarrow\compute.py:198: RuntimeWarning: Python binding for RankOptions not exposed
  .format(class_name), RuntimeWarning)


In [2]:
df.head()


,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


In [3]:
df = df.drop(['article_link'], axis=1)#Droping the article_link coloum 
df.head() 

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0


In [4]:
X = df['headline']

In [5]:
X

0        former versace store clerk sues over secret 'b...
1        the 'roseanne' revival catches up to our thorn...
2        mom starting to fear son's web series closest ...
3        boehner just wants wife to listen, not come up...
4        j.k. rowling wishes snape happy birthday in th...
                               ...                        
26704                 american politics in moral free-fall
26705                              america's best 20 hikes
26706                                reparations and obama
26707    israeli ban targeting boycott supporters raise...
26708                    gourmet gifts for the foodie 2014
Name: headline, Length: 26709, dtype: object

In [6]:
Y = df['is_sarcastic']

In [7]:
Y

0        0
1        0
2        1
3        1
4        0
        ..
26704    0
26705    0
26706    0
26707    0
26708    0
Name: is_sarcastic, Length: 26709, dtype: int64

In [8]:
import tensorflow  as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization  

In [9]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 10 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [10]:
text_vectorizer.adapt(X)


In [11]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

#

In [12]:
#functional Aip 
#model creation
tf.random.set_seed(42)
from tensorflow.keras import layers
Inputs =  layers.Input(shape=(1,),dtype='string')
x = text_vectorizer(Inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(3,activation='relu')(x)
#x = layers.Dropout(0.2)(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
Dense_model = tf.keras.Model(Inputs,outputs,name='Dens_model')

In [13]:
#model compile 
Dense_model.compile(loss="binary_crossentropy",
                    optimizer =tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])

In [14]:
from sklearn.model_selection import KFold

n_split=3

for train_index,test_index in KFold(n_split).split(X):
  x_train,x_test=X[train_index],X[test_index]
  y_train,y_test=Y[train_index],Y[test_index]

  Dense_model.fit(x_train, y_train,epochs=30)

Epoch 1/30
557/557 [==============================] - 3s 3ms/step - loss: 0.4697 - accuracy: 0.7853
Epoch 2/30
557/557 [==============================] - 2s 3ms/step - loss: 0.2671 - accuracy: 0.8925
Epoch 3/30
557/557 [==============================] - 2s 3ms/step - loss: 0.1959 - accuracy: 0.9252
Epoch 4/30
557/557 [==============================] - 2s 3ms/step - loss: 0.1531 - accuracy: 0.9455
Epoch 5/30
557/557 [==============================] - 2s 3ms/step - loss: 0.1234 - accuracy: 0.9552
Epoch 6/30
557/557 [==============================] - 2s 3ms/step - loss: 0.1026 - accuracy: 0.9639
Epoch 7/30
557/557 [==============================] - 2s 3ms/step - loss: 0.0847 - accuracy: 0.9703
Epoch 8/30
557/557 [==============================] - 2s 3ms/step - loss: 0.0710 - accuracy: 0.9752
Epoch 9/30
557/557 [==============================] - 2s 3ms/step - loss: 0.0589 - accuracy: 0.9800
Epoch 10/30
557/557 [==============================] - 1s 3ms/step - loss: 0.0512 - accuracy: 0.9820

557/557 [==============================] - 2s 3ms/step - loss: 0.0491 - accuracy: 0.9677
Epoch 23/30
557/557 [==============================] - 2s 3ms/step - loss: 0.0490 - accuracy: 0.9680
Epoch 24/30
557/557 [==============================] - 1s 3ms/step - loss: 0.0490 - accuracy: 0.9678
Epoch 25/30
557/557 [==============================] - 2s 3ms/step - loss: 0.0495 - accuracy: 0.9679
Epoch 26/30
557/557 [==============================] - 2s 3ms/step - loss: 0.0487 - accuracy: 0.9680
Epoch 27/30
557/557 [==============================] - 2s 3ms/step - loss: 0.0488 - accuracy: 0.9678
Epoch 28/30
557/557 [==============================] - 2s 3ms/step - loss: 0.0484 - accuracy: 0.9684
Epoch 29/30
557/557 [==============================] - 2s 3ms/step - loss: 0.0483 - accuracy: 0.9682
Epoch 30/30
557/557 [==============================] - 2s 3ms/step - loss: 0.0483 - accuracy: 0.9681


In [16]:
predictions = Dense_model.predict(x_test)

279/279 [==============================] - 1s 2ms/step


In [25]:
print(predictions)

[[9.9948210e-01]
 [9.9999905e-01]
 [4.5431394e-04]
 ...
 [1.1259835e-04]
 [3.6183323e-10]
 [1.6890378e-10]]


In [29]:
import numpy as np

In [38]:
# Get mean pred probs for 3 models
baseline_pred_probs = np.max(Dense_model.predict(x_test), axis=1) # get the prediction probabilities from baseline model
predictions=tf.round(tf.squeeze(baseline_pred_probs))

279/279 [==============================] - 0s 2ms/step


In [27]:
print(tf.squeeze(predictions))

tf.Tensor(
[9.9948210e-01 9.9999905e-01 4.5431394e-04 ... 1.1259835e-04 3.6183323e-10
 1.6890378e-10], shape=(8903,), dtype=float32)


In [21]:
print('Model evaluation ',Dense_model.evaluate(x_test,y_test))

279/279 [==============================] - 1s 2ms/step - loss: 0.4356 - accuracy: 0.9134
Model evaluation  [0.43564096093177795, 0.9133999943733215]


In [22]:
print('Model evaluation ',Dense_model.evaluate(x_train,y_train))

557/557 [==============================] - 1s 2ms/step - loss: 0.0478 - accuracy: 0.9686
Model evaluation  [0.04775463044643402, 0.9686061143875122]


In [39]:
from sklearn.metrics import f1_score

print("F1 Score: {}".format(f1_score(y_test,predictions)))

F1 Score: 0.9023432552248257


In [40]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.94      0.91      0.92      5037
           1       0.88      0.92      0.90      3866

    accuracy                           0.91      8903
   macro avg       0.91      0.91      0.91      8903
weighted avg       0.91      0.91      0.91      8903



In [44]:
Dense_model.save('sarcastic_Dense',save_format='tf')

INFO:tensorflow:Assets written to: sarcastic_Dense\assets


In [47]:
df[]

,headline,is_sarcastic
0,former versace store clerk sues over secret 'b...,0
1,the 'roseanne' revival catches up to our thorn...,0
2,mom starting to fear son's web series closest ...,1
3,"boehner just wants wife to listen, not come up...",1
4,j.k. rowling wishes snape happy birthday in th...,0
...,...,...
26704,american politics in moral free-fall,0
26705,america's best 20 hikes,0
26706,reparations and obama,0
26707,israeli ban targeting boycott supporters raise...,0
